In [2]:
import datasets

knowledge_base = datasets.load_dataset("m-ric/huggingface_doc", split="train")
filtered_knowledge_base = knowledge_base.filter(
    lambda row: row["source"].startswith("huggingface/transformers")
)

d:\projects\rag-notebooks\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.retrievers import BM25Retriever

source_docs = [
    Document(page_content=doc["text"], metadata={"source": doc["source"].split("/")[1]})
    for doc in filtered_knowledge_base
]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    add_start_index=True,
    strip_whitespace=True,
    separators=["\n\n", "\n", ".", " ", ""],
)
docs_processed = text_splitter.split_documents(source_docs)

from smolagents import Tool


class RetrieverTool(Tool):
    name = "retriever"
    description = "Uses semantic search to retrieve the parts of transformers documentation that could be most relevant to answer your query."
    inputs = {
        "query": {
            "type": "string",
            "description": "The query to perform. This should be semantically close to your target documents. Use the affirmative form rather than a question.",
        }
    }
    output_type = "string"

    def __init__(self, docs, **kwargs):
        super().__init__(**kwargs)
        self.retriever = BM25Retriever.from_documents(docs, k=10)

    def forward(self, query: str) -> str:
        assert isinstance(query, str), "Your search query must be a string"

        docs = self.retriever.invoke(
            query,
        )
        return "\nRetrieved documents:\n" + "".join(
            [
                f"\n\n===== Document {str(i)} =====\n" + doc.page_content
                for i, doc in enumerate(docs)
            ]
        )


from smolagents import HfApiModel, CodeAgent, LiteLLMModel

model_id = "ollama/qwen2.5-coder:14b"
model = LiteLLMModel(model_id=model_id, api_base="http://localhost:11434")

retriever_tool = RetrieverTool(docs_processed)
agent = CodeAgent(
    tools=[retriever_tool],
    model=model,
    max_steps=4,
    verbosity_level=2,
    add_base_tools=True,
)

agent_output = agent.run(
    "For a transformers model training, which is slower, the forward or the backward pass?"
)

print("Final output:")
print(agent_output)

d:\projects\rag-notebooks\.venv\Lib\site-packages\pydantic\_internal\_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


╭────────────────────────────────────────────────────────────────────────────────────────────── New run ───────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                                                                                      │
│ For a transformers model training, which is slower, the forward or the backward pass?                                                                                                                │
│                                                                                                                                                                                                      │
╰─ LiteLLMModel - ollama/qwen2.5-coder:14b ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Thought: To determine whether the forward or backward pass in Transformers model training is slower, I need to first understand what these terms mean in the context of neural network training. The    
forward pass involves passing input data through the model to get predictions, while the backward pass (also known as backpropagation) involves computing gradients to update the model weights.        
                                                                                                                                                                                                        
To get a detailed understanding and comparison, I will use the `retriever` tool to find relevant information from the Transformers documentation.                                                       
                                                                                                                                                                                                        
Code:                                                                                                                                                                                                   
```py                                                                                                                                                                                                   
forward_backward_info = retriever(query="Transformer forward and backward pass")                                                                                                                        
print("Information about Transformer forward and backward pass:", forward_backward_info)                                                                                                                
```                                                                                                                                                                                                     

─ Executing this code: ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── 
  forward_backward_info = retriever(query="Transformer forward and backward pass")                                                                                                                      
  print("Information about Transformer forward and backward pass:", forward_backward_info)                                                                                                              
 ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

...

[Step 3: Duration 15.21 seconds| Input tokens: 8,192 | Output tokens: 658]

Reached max steps.

Final answer: In transformer models, the backward pass is typically slower than the forward pass. The backward pass involves calculating gradients through backpropagation, which can be computationally
intensive and time-consuming compared to the forward pass, where only inference is performed. The provided documents indicate that both training and inference times are measured in benchmarks, with 
training time encompassing both forward and backward passes.

[Step 4: Duration 0.00 seconds| Input tokens: 10,240 | Output tokens: 732]

Final output:
In transformer models, the backward pass is typically slower than the forward pass. The backward pass involves calculating gradients through backpropagation, which can be computationally intensive and time-consuming compared to the forward pass, where only inference is performed. The provided documents indicate that both training and inference times are measured in benchmarks, with training time encompassing both forward and backward passes.
